In [1]:
import pandas as pd
import numpy as np

## Exercise 13*
`data/book_extract.csv` file consists of records regarding financial performance of dealers. Prepare a report which explicates financial performance of dealers eligible for a payable bonus based on net result (`income-(clawback+opex)`) for each calendar month, given that opex/income ratio does not exceed 15%.

Report should have tabular form, rows indicating bonus basis for each month with at least one eligible dealer, columns indicating all dealers.

In [2]:
#wczytywanie tabeli
book_extract = pd.read_csv("data/book_extract.csv")
book_extract

,Unnamed: 0,name,date,operation type,amount
0,0,Marie Pugh,2020-05-08,income,107.513913
1,1,Havin Cope,2020-04-19,clawback,147.318495
2,2,Havin Cope,2020-01-07,opex,8.189630
3,3,Havin Cope,2020-03-14,income,349.687260
4,4,Marie Pugh,2020-03-16,opex,76.655636
...,...,...,...,...,...
995,995,Havin Cope,2020-03-04,clawback,895.713291
996,996,Frazer Bone,2020-09-13,clawback,31.235687
997,997,Havin Cope,2020-06-07,income,437.871498
998,998,Havin Cope,2020-02-17,opex,8.111265


In [3]:
##Tworzymy kopię zmienionej tabeli book_extract
##usuwamy wiersz z indeksem i zamieniamy datę na nr_miesiąca
book_extract_copy = book_extract.copy()
book_extract_copy = book_extract_copy.drop(book_extract.columns[[0]], axis=1)

book_extract_copy['month'] = pd.DatetimeIndex(book_extract_copy['date']).month
book_extract_copy = book_extract_copy.drop(book_extract.columns[[2]], axis=1)
book_extract_copy['name'].unique()
book_extract_copy

,name,operation type,amount,month
0,Marie Pugh,income,107.513913,5
1,Havin Cope,clawback,147.318495,4
2,Havin Cope,opex,8.189630,1
3,Havin Cope,income,349.687260,3
4,Marie Pugh,opex,76.655636,3
...,...,...,...,...
995,Havin Cope,clawback,895.713291,3
996,Frazer Bone,clawback,31.235687,9
997,Havin Cope,income,437.871498,6
998,Havin Cope,opex,8.111265,2


In [4]:
## dodajemy wiersze ze wszystkimi miesiącami i przychodem 0, gdyby ktoś nie miał finansów w którymś miesiącu
lista = ['income', 'clawback', 'opex']
names = book_extract_copy['name'].unique()

book_extract_copy = book_extract_copy.to_numpy()

for nm in names:
    for nr_month in range(12):
        for el in lista:
            ##1. dealers -> do  dodania!!!
            #book_extract_copy = pd.concat(book_extract_copy, new_row)
            #book_extract_copy = book_extract_copy.append(new_row, ignore_index=True)
            new_row = [nm, el, 0, nr_month+1]
            book_extract_copy = np.append(book_extract_copy, [new_row], axis=0)
            #new_row = pd.DataFrame.from_records(new_row)
            
book_extract_copy = pd.DataFrame(book_extract_copy)
type(book_extract_copy)
book_extract_copy.columns = ['name', 'operation type', 'amount', 'month']
book_extract_copy["month"] = book_extract_copy["month"].astype('int')
book_extract_copy["amount"] = book_extract_copy["amount"].astype('float32')
book_extract_copy

,name,operation type,amount,month
0,Marie Pugh,income,107.513916,5
1,Havin Cope,clawback,147.318497,4
2,Havin Cope,opex,8.189630,1
3,Havin Cope,income,349.687256,3
4,Marie Pugh,opex,76.655640,3
...,...,...,...,...
1175,Iyla Tate,clawback,0.000000,11
1176,Iyla Tate,opex,0.000000,11
1177,Iyla Tate,income,0.000000,12
1178,Iyla Tate,clawback,0.000000,12


In [5]:
#pogrupowana tabela z sumą przychodów
book_extract_copy = book_extract_copy.groupby(['name', 'operation type', 'month']).sum()
to_excel = book_extract_copy.to_excel("data/groups.xlsx", sheet_name='after groupby')
book_extract_copy

amount
name        operation type month             
Frazer Bone clawback       1         0.000000
                           2       986.680298
                           3      1081.955078
                           4       238.083633
                           5       454.923431
...                                       ...
Marie Pugh  opex           8       271.334839
                           9         0.000000
                           10        0.000000
                           11        0.000000
                           12        0.000000

[180 rows x 1 columns]

In [6]:
##tworzymy tabelę z imionami:
dealers_names = book_extract.drop(book_extract.columns[[0, 2, 3, 4]], axis=1)
dealers_names = dealers_names.groupby(['name']).sum()
dealers_names

""
name
Frazer Bone
Grady Kearney
Havin Cope
Iyla Tate
Marie Pugh


In [7]:
##GŁÓWNA CZĘŚĆ
#tworzenie nowej tabeli 

for k in range(12): #12 miesiecy
    listu = np.zeros(5, dtype=np.float32)
    
    num = 0 #liczymy liczbę niepustych
    for n in range(5): #5 osób
        start = 36 * n + k
        #val = income-(clawback+opex)
        #wybieramy komórki z book_extract_copy odpowiadające income, clawback, opex
        val = book_extract_copy["amount"][start + 12] - (book_extract_copy["amount"][start] + book_extract_copy["amount"][start + 24])
        
        #sprawdzamy, kto ma opex/income < .15
        if book_extract_copy["amount"][start + 12] == 0:
            listu[n] = np.nan
        elif book_extract_copy["amount"][start + 24] / book_extract_copy["amount"][start + 12] >= 0.15:
            listu[n] = np.nan
        else:
            listu[n] = val  
            num += 1
    
    #dodajemy tylko niepuste kolumny:
    if num > 0:
        dealers_names[str(k + 1)] = listu

In [8]:
print("dealers eligible for a payable bonus (with opex/income < .15):")
dealers_names

dealers eligible for a payable bonus (with opex/income < .15):


,1,3,4,6,11,12
name,,,,,,
Frazer Bone,NaN,NaN,NaN,NaN,NaN,NaN
Grady Kearney,2760.968506,2565.874023,2506.448242,NaN,NaN,NaN
Havin Cope,NaN,2963.592285,NaN,NaN,NaN,NaN
Iyla Tate,NaN,NaN,710.488037,NaN,3328.786865,844.858459
Marie Pugh,162.244873,2990.475342,NaN,1875.947876,NaN,NaN
